In [1]:
# Base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Plot settings

%matplotlib inline
sns.set_context('talk')
sns.set_palette('gray')
sns.set_style('ticks', {'grid.color' : '0.9'})

In [3]:
# Algorithms

from sklearn.ensemble import RandomForestClassifier

In [4]:
# Model selection

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score

In [5]:
x_train = np.load('processed/wells_feature_matrix.npy')
y_train = pd.read_csv('processed/wells_labels_train.csv')['status'].tolist()

We begin by looking at the variation in the performance as we hand-tune some of the parameters. We will then put together a parameter grid that will possiblly find the best combination among all the parameters. For now, we will look at accuracy as the metric. At the moment the focus is on observing the contours of accuracy as we change the hyperparameters

### Number of estimators

In [ ]:
n_estimators_list = [50, 100, 500, 1000]

In [ ]:
rf_classif = RandomForestClassifier(n_estimators=50,
                                    max_features='auto',
                                    random_state=20130810,
                                    n_jobs=3)

In [ ]:
rf_cv = GridSearchCV(estimator=rf_classif, 
                     param_grid={'n_estimators' : n_estimators_list},
                     scoring='accuracy',
                     n_jobs=3)

In [ ]:
%%time
rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv.cv_results_['mean_test_score']

### Maximum depth of the tree

In [ ]:
np.sqrt(x_train.shape[1])

In [ ]:
max_depth_list = [5, 15, 30, 50, 100]

In [ ]:
rf_classif = RandomForestClassifier(n_estimators=50,
                                    max_features='auto',
                                    random_state=20130810,
                                    n_jobs=3)

In [ ]:
rf_cv = GridSearchCV(estimator=rf_classif, 
                     param_grid={'max_depth' : max_depth_list},
                     scoring='accuracy',
                     n_jobs=3)

In [ ]:
%%time
rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv.cv_results_['mean_test_score']

### Maximum number of features to consider at each split 

In [ ]:
max_features_list = [5, 15, 30, 50]

In [ ]:
rf_classif = RandomForestClassifier(n_estimators=50,
                                    max_features='auto',
                                    random_state=20130810,
                                    n_jobs=3)

In [ ]:
rf_cv = GridSearchCV(estimator=rf_classif, 
                     param_grid={'max_features' : max_features_list},
                     scoring='accuracy',
                     n_jobs=3)

In [ ]:
%%time
rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv.cv_results_['mean_test_score']

### Minimum samples to split an internal node

In [ ]:
min_samples_split_list = [2, 5, 10, 20]

In [ ]:
rf_classif = RandomForestClassifier(n_estimators=50,
                                    max_features='auto',
                                    random_state=20130810,
                                    n_jobs=3)

In [ ]:
rf_cv = GridSearchCV(estimator=rf_classif, 
                     param_grid={'min_samples_split' : min_samples_split_list},
                     scoring='accuracy',
                     n_jobs=3)

In [ ]:
%%time
rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv.cv_results_['mean_test_score']

### Minimum samples to be a leaf node

In [ ]:
min_samples_leaf_list = [1, 5, 10, 20]

In [ ]:
rf_classif = RandomForestClassifier(n_estimators=50,
                                    max_features='auto',
                                    random_state=20130810,
                                    n_jobs=3)

In [ ]:
rf_cv = GridSearchCV(estimator=rf_classif, 
                     param_grid={'min_samples_leaf' : min_samples_leaf_list},
                     scoring='accuracy',
                     n_jobs=3)

In [ ]:
%%time
rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv.cv_results_['mean_test_score']

### Randomized grid search over parameter combinations

Now we tune the random forests model over a parameter grid

In [6]:
pgrid = {'min_samples_leaf' : [5, 10],
         'min_samples_split' : [5, 10],
         'max_features' : [15, 30],
         'max_depth' : [30, 50 , 100],
         'n_estimators' : [100, 500]}

In [7]:
rf_classif = RandomForestClassifier(random_state = 20130810,
                                    n_jobs = 3)

In [8]:
rf_cv = RandomizedSearchCV(estimator = rf_classif,
                           param_distributions = pgrid,
                           n_iter = 10,
                           cv = 5,
                           n_jobs = 3, 
                           random_state = 20130810,
                           verbose = 2)

In [9]:
%%time
rf_cv.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  8.8min
[Parallel(n_jobs=3)]: Done  50 out of  50 | elapsed: 13.8min finished


Wall time: 14min 44s


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=False, random_state=20130810, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=3,
          param_distributions={'min_samples_leaf': [5, 10], 'min_samples_split': [5, 10], 'max_features': [15, 30], 'max_depth': [30, 50, 100], 'n_estimators': [100, 500]},
          pre_dispatch='2*n_jobs', random_state=20130810, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [11]:
rf_cv.best_score_

0.8004377104377104

In [10]:
rf_cv.best_params_

{'max_depth': 50,
 'max_features': 30,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 500}

### Running the model at the final configurations

In [16]:
rf_classif = RandomForestClassifier(n_estimators=500,
                                    max_depth=50,
                                    max_features=30,
                                    min_samples_leaf=5,
                                    min_samples_split=10,
                                    random_state=20130810,
                                    n_jobs=3)

In [19]:
scores = cross_val_score(rf_classif,
                         x_train,
                         y_train, 
                         scoring = 'accuracy',
                         cv = 10,
                         n_jobs = 3,
                         verbose = 3)

[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  8.1min finished


In [21]:
scores.mean(), scores.std()

(0.804461299469365, 0.004635098621363507)